In [2]:
origin="drive/MyDrive/twitter/onechunk/"

#Prepare speaks language feature


##Setup

In [3]:
import os   

In [9]:
os.system(f"cp {origin}* ../content/")

0

In [10]:
!bunzip2 train.bz2
!mv train train.csv
!bunzip2 test.bz2
!mv test test.csv

In [15]:
import pandas as pd
train=pd.read_csv("train.csv", index_col=0)
test=pd.read_csv("test.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
import gc
gc.collect()

240

In [17]:
def createTempTable(df,isTrain=True):
    t=df[["engaging_user_id","language","interaction"]].copy()
    lang=t
    langDir=lang[lang["interaction"]>0]
    langDir.drop("interaction",inplace=True,axis=1)
    langDir.columns=["id","language"]
    t=train[["engaged_with_user_id","language","interaction"]].copy()
    lang=t
    langInv=lang[lang["interaction"]>0]
    langInv.drop("interaction",inplace=True,axis=1)
    langInv.columns=["id","language"]
    lang=pd.concat([langDir,langInv])
    del langDir
    del langInv
    lang.insert(2,"understands",1)
    lang.columns=["engaging_user_id","language","understands"]
    lang=lang.groupby(["engaging_user_id","language"]).sum()
    lang["understands"]=lang["understands"].apply(lambda x: (x>1) if isTrain else x>0)
    temp=lang[lang["understands"]].copy()
    del lang
    return temp
    



In [18]:
langTrain=createTempTable(train)
langTest=createTempTable(train,isTrain=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.p

In [19]:
train=train.merge(langTrain,on=["engaging_user_id","language"],how="left")
test=test.merge(langTest,on=["engaging_user_id","language"],how="left")

##Fill if no information about language knowledge (##TODO fill with prior)


In [20]:
train["understands"]=train["understands"].fillna(False)
test["understands"]=test["understands"].fillna(False)
del langTrain
del langTest

##Drop Language

In [22]:
train.drop("language",axis=1,inplace=True)
test.drop("language",axis=1,inplace=True)

#Graph features

Choose dataframe to which add features

In [88]:
#df=train
df=test 

##ADD pagerank

In [89]:
pagerank_ed_to_ing=pd.read_csv("pagerank.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [90]:
pagerank_ed_to_ing.columns=["pagerank_ed_to_ing","engaged_with_user_id"]

In [91]:
pagerank_ed_to_ing[:5]

,pagerank_ed_to_ing,engaged_with_user_id
1131207,0.000280,1DE5D18797E212A3850C5FB5C35E12E2
1519173,0.000207,B4246E7A34B52630ED396609E307F784
140261,0.000189,9C3AAFC3C2C2F129065F4BB5B00426B4
1511093,0.000184,F050BCE6C6A405C11C8D7897017CE1C1
1174528,0.000171,D956CB05C8566B63F75C86A595A84417


In [92]:
df=df.merge(pagerank_ed_to_ing,on="engaged_with_user_id",how="left")

In [93]:
df["pagerank_ed_to_ing_ed"]=df["pagerank_ed_to_ing"].apply(lambda x: x if x>0 else 0)
df.drop("pagerank_ed_to_ing",axis=1,inplace=True)

In [94]:
pagerank_ed_to_ing.columns=["pagerank_ed_to_ing","engaging_user_id"]

In [95]:
df=df.merge(pagerank_ed_to_ing,on="engaging_user_id",how="left")

In [96]:
df["pagerank_ed_to_ing_ing"]=df["pagerank_ed_to_ing"].apply(lambda x: x if x>0 else 0)
df.drop("pagerank_ed_to_ing",axis=1,inplace=True)


In [97]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00


In [98]:
del pagerank_ed_to_ing

##ADD HITS

In [99]:
hits_ed_to_ing=pd.read_csv("hits.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [100]:
hits_ed_to_ing.columns=["hubs_ed","authorities_ed","engaged_with_user_id"]

In [101]:
hits_ed_to_ing[:5]

,hubs_ed,authorities_ed,engaged_with_user_id
0,1.165709e-09,1.165716e-09,02FA3E768BA51EE08AA9D098ED079482
1,3.879835e-12,3.879881e-12,B47FCA079E5D01F901B819ECE263F1DD
2,0.000000e+00,0.000000e+00,E36A5DA720EF476EC61F79B753CCEDA5
3,7.597836e-14,7.598496e-14,15FCC988F9C819109E20D31BE923A578
4,2.612996e-11,2.613013e-11,98C1837F4BAA52B93B45E0AF8C375C2F


In [102]:
df=df.merge(hits_ed_to_ing,on="engaged_with_user_id",how="left")

In [103]:
df["hubs_ed"]=df["hubs_ed"].apply(lambda x: x if x>0 else 0)
df["authorities_ed"]=df["authorities_ed"].apply(lambda x: x if x>0 else 0)

In [104]:
hits_ed_to_ing.columns=["hubs_ing","authorities_ing","engaging_user_id"]

In [105]:
df=df.merge(hits_ed_to_ing,on="engaging_user_id",how="left")

In [106]:
df["hubs_ing"]=df["hubs_ing"].apply(lambda x: x if x>0 else 0)
df["authorities_ing"]=df["authorities_ing"].apply(lambda x: x if x>0 else 0)

In [107]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07,5.550088e-23,5.550078e-23,0.000000e+00,0.000000e+00
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07,2.493393e-23,2.493290e-23,1.414449e-20,1.414458e-20
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07,6.101975e-15,6.101506e-15,2.555951e-09,2.555959e-09
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [108]:
del hits_ed_to_ing

##ADD Katz

In [109]:
katz_ed_to_ing=pd.read_csv("katz_centrality.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [110]:
katz_ed_to_ing.columns=["katz_ed","engaged_with_user_id"]

In [111]:
katz_ed_to_ing[:5]

,katz_ed,engaged_with_user_id
0,0.001637,D929B501E45AABD4E2AF47745D272157
1,0.004231,6FADF44AFDC89F764E2AF553E814A3AD
2,0.001586,CE02EC14E2D64DF46EF87B2E0244B415
3,0.003249,FB35EA040622F3C04AE9E1EA52147FE6
4,0.001755,C6149C2908123D272325E5C0D67F5BF0


In [112]:
df=df.merge(katz_ed_to_ing,on="engaged_with_user_id",how="left")

In [113]:
df["katz_ed"]=df["katz_ed"].apply(lambda x: x if x>0 else 0)

In [114]:
katz_ed_to_ing.columns=["katz_ing","engaging_user_id"]

In [115]:
df=df.merge(katz_ed_to_ing,on="engaging_user_id",how="left")

In [116]:
df["katz_ing"]=df["katz_ing"].apply(lambda x: x if x>0 else 0)

In [117]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07,5.550088e-23,5.550078e-23,0.000000e+00,0.000000e+00,0.000528,0.000176
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07,2.493393e-23,2.493290e-23,1.414449e-20,1.414458e-20,0.000176,0.000536
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07,6.101975e-15,6.101506e-15,2.555951e-09,2.555959e-09,0.002286,0.000356
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000


In [118]:
del katz_ed_to_ing

##ADD Overlap

In [119]:
overlap_ed_to_ing=pd.read_csv("over.csv",index_col=0)


In [120]:
overlap_ed_to_ing.columns=["overlap_ed","engaged_with_user_id","engaging_user_id"]

In [121]:
overlap_ed_to_ing[:5]

,overlap_ed,engaged_with_user_id,engaging_user_id
9431,0.666667,56238EC8CB5C51DA10E228C7D4D395A4,2BE23D1A601A50E2D007C6F0090CF091
15942,0.666667,DBA3D392CFA98ED80D4D35CFFAFAB0CB,66A770D1BE1D360AF2A33F024284295E
120536,0.666667,CA33EBA30174D470C5A2189D96D668F3,7B6A39F2FE6D377A75F857E1837DD8B8
148176,0.666667,2BE23D1A601A50E2D007C6F0090CF091,56238EC8CB5C51DA10E228C7D4D395A4
178048,0.666667,93207183C9BFD46714F749AF05C0246A,B474FE65F1C6025D8DED4D3C5FADC11E


In [122]:
df=df.merge(overlap_ed_to_ing,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [123]:
df["overlap_ed"]=df["overlap_ed"].apply(lambda x: x if x>0 else 0)


In [124]:
overlap_ed_to_ing.columns=["overlap_ing","engaging_user_id","engaged_with_user_id"]

In [125]:
df=df.merge(overlap_ed_to_ing,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [126]:
df["overlap_ing"]=df["overlap_ing"].apply(lambda x: x if x>0 else 0)


In [127]:
def f(row):
    e=row["overlap_ed"]
    i=row["overlap_ing"]
    if e >0:
        if i>0:
            return (e+i)/2
        else:
            return e
    else:
        if i>0:
            return i
        else:
            return 0
df["overlap"]=df.apply(f, axis=1)

df.drop("overlap_ed",axis=1,inplace=True)
df.drop("overlap_ing",axis=1,inplace=True)

In [128]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,overlap
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07,5.550088e-23,5.550078e-23,0.000000e+00,0.000000e+00,0.000528,0.000176,0.0
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07,2.493393e-23,2.493290e-23,1.414449e-20,1.414458e-20,0.000176,0.000536,0.0
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07,6.101975e-15,6.101506e-15,2.555951e-09,2.555959e-09,0.002286,0.000356,0.0
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0


In [129]:
del overlap_ed_to_ing

##ADD Jaccard

###ADD Jaccard to the dataframe

In [130]:
jaccard_ed_to_ing=pd.read_csv("jaccard-ed-to-ing.csv",index_col=0)


In [131]:
jaccard_ed_to_ing.columns=["jaccard_ed_to_ing_ed","engaged_with_user_id","engaging_user_id"]

In [132]:
jaccard_ed_to_ing[:5]

,jaccard_ed_to_ing_ed,engaged_with_user_id,engaging_user_id
713697,0.666667,5CB68E06FAF45602C095AE5964D2B4C2,1E310376604254152F0DDC60D0093347
2284025,0.666667,1E310376604254152F0DDC60D0093347,5CB68E06FAF45602C095AE5964D2B4C2
554994,0.600000,DAA3B34033B99FC4BCCF456EEB073D40,FAE7207C477F9B680174EDC501A76C44
1829080,0.600000,FAE7207C477F9B680174EDC501A76C44,DAA3B34033B99FC4BCCF456EEB073D40
458055,0.500000,67ECB116DEC459EFD4C8F1199370BE17,7D8560C4125CC618C34515689DD8D9E4


In [133]:
df=df.merge(jaccard_ed_to_ing,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [134]:
df["jaccard_ed_to_ing_ed"]=df["jaccard_ed_to_ing_ed"].apply(lambda x: x if x>0 else 0)


In [135]:
jaccard_ed_to_ing.columns=["jaccard_ed_to_ing_ing","engaging_user_id","engaged_with_user_id"]

In [136]:
df=df.merge(jaccard_ed_to_ing,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [137]:
df["jaccard_ed_to_ing_ing"]=df["jaccard_ed_to_ing_ing"].apply(lambda x: x if x>0 else 0)


In [138]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,overlap,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07,5.550088e-23,5.550078e-23,0.000000e+00,0.000000e+00,0.000528,0.000176,0.0,0.0,0.0
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07,2.493393e-23,2.493290e-23,1.414449e-20,1.414458e-20,0.000176,0.000536,0.0,0.0,0.0
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07,6.101975e-15,6.101506e-15,2.555951e-09,2.555959e-09,0.002286,0.000356,0.0,0.0,0.0
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0


In [139]:
del jaccard_ed_to_ing

###ADD Jaccard to the dataframe opposite

In [140]:
jaccard_ing_to_ed=pd.read_csv("jaccard-ing-to-ed.csv",index_col=0)


In [141]:
jaccard_ing_to_ed.columns=["jaccard_ing_to_ed_ed","engaged_with_user_id","engaging_user_id"]

In [142]:
jaccard_ing_to_ed[:5]

,jaccard_ing_to_ed_ed,engaged_with_user_id,engaging_user_id
263536,0.666667,5BCD16926EDF8B3A66E7D658AC70513D,06C5FD59BD965EE20CBA4F7D4D7F7859
280054,0.666667,69B871CC7FEE3B5381941B7F71006C82,EC478D67017FCEF8CA658CA3CECE2C47
306050,0.666667,723DB181D6E6B38379B8B53E2C098ECC,62AF7A8F6A3DA4B3FDE7D0FBDE1477FA
917929,0.666667,8C69FBD2209B0F1DE08ABB50BBADEF6F,B3733C09AD13E0B01D0134FB3CCAFE44
1321244,0.666667,2641FBB1227B81448E9432B2823BFB7E,CD33A2E2CC5284196899F5CC87E74D11


In [143]:
df=df.merge(jaccard_ing_to_ed,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [144]:
df["jaccard_ing_to_ed_ed"]=df["jaccard_ing_to_ed_ed"].apply(lambda x: x if x>0 else 0)


In [145]:
jaccard_ing_to_ed.columns=["jaccard_ing_to_ed_ing","engaging_user_id","engaged_with_user_id"]

In [146]:
df=df.merge(jaccard_ing_to_ed,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [147]:
df["jaccard_ing_to_ed_ing"]=df["jaccard_ing_to_ed_ing"].apply(lambda x: x if x>0 else 0)


In [148]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,understands,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,overlap,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing
0,78,0,0,1,1,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0,0,0,1,True,9.109596e-07,4.404669e-07,5.550088e-23,5.550078e-23,0.000000e+00,0.000000e+00,0.000528,0.000176,0.0,0.0,0.0,0.0,0.0
1,77,0,0,1,1,02EA65880A80121A394AB9CD652D7AD6,881,583,False,D3DE61AA5DFB88CEFBC2C9E675F40723,355,650,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.341039e-07,8.432427e-07,2.493393e-23,2.493290e-23,1.414449e-20,1.414458e-20,0.000176,0.000536,0.0,0.0,0.0,0.0,0.0
2,62,0,0,0,0,0A114052FE4DD3F5F7D2589673EC5ED8,133933,5371,False,0D07725F473577EE50C4EE831FC66D99,83,265,False,False,1,0,0,0,1,0,0,0,0,0,1,True,3.882743e-06,5.928389e-07,6.101975e-15,6.101506e-15,2.555951e-09,2.555959e-09,0.002286,0.000356,0.0,0.0,0.0,0.0,0.0
3,19,0,0,0,0,CD405E59462653553D5694E04F64A65D,234,409,False,2E6002102712D8815CAB8DE1C63CC0BB,24,138,False,True,1,0,0,1,2,0,0,0,1,0,0,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,32,0,1,0,0,9ECFCC34E0AFB2B0A025BA64FA42BF50,357,474,False,2C77DE8BEFE86F6FC7C3A2741B9AC961,407,416,False,True,0,0,0,0,0,1,0,0,0,0,1,False,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [149]:
del jaccard_ing_to_ed

#Drop user ids after adding graph features

In [150]:
df.drop("engaged_with_user_id",axis=1,inplace=True)
df.drop("engaging_user_id",axis=1,inplace=True)
df.drop("interaction",axis=1,inplace=True)

#Save Complete dataframe

In [151]:
#df.to_csv("trainFinal.csv", header=df.columns);del train
df.to_csv("testFinal.csv", header=df.columns);del test
del df

In [152]:
!cp trainFinal.csv  drive/MyDrive/twitter/onechunk/trainFinal.csv
!cp testFinal.csv  drive/MyDrive/twitter/onechunk/testFinal.csv